# Packet Delivery Agent

In [233]:
import random
import math
import numpy as np

### Subset generation using simulated annealing

In [234]:
# Method to generate random sample from an array
def generate_random_solution(weights, length):
    return sorted(random.sample(weights, length))


# Method to generate all possible neighbours
def get_neighbourhood_subsets(weights, subset):
    neighbourhood = []
    
    # neighbours with one element less
    if len(subset) > 1:
        for index in range(len(subset)):
            neighbourhood.append(subset[:index] + subset[index + 1:])

    # neighbours with one element more
    # subset_set = set(subset)
    if len(subset) < len(weights):
        for item in weights:
            if item not in subset:
                neighbourhood.append(subset + [item])
    
    # replace every element in subset with one element from set
    for index in range(len(subset)):
        for set_item in weights:
            if set_item not in subset:
                neighbourhood.append(subset[:index] + [set_item] + subset[index + 1:])
    return neighbourhood


# Helper method to determine whether random_subset is better than
# normal subset. Returns True if random_subset is better, False
# otherwise
def is_better_neighbour(subset, random_subset):
    if random_subset:
        return abs(sum(random_subset)) < abs(sum(subset))
    return False

# Method for performing the simulated_annealing for generating subset which has sum 10
def simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations):
    if sum(subset) == target_sum:
        return best_subset
    if initial_temp <= final_temp:
        return best_subset
    if iterations == max_iterations:
        return best_subset
    if len(subset) == 1: # weights
        return subset
    
    #print(f"Iteration: {iterations}")
    
    neighbourhood_subsets = get_neighbourhood_subsets(weights, subset)
    # print(f"Neighbours: {neighbourhood_subsets}")
    # print(f"Total number of Neighbours: {len(neighbourhood_subsets)}")
    
    random_neighbour = random.choice(neighbourhood_subsets)
    # print(f"Random neighbour chosen: {random_neighbour} of sum: {sum(random_neighbour)}")
    
    if is_better_neighbour(subset, random_neighbour):
        # print('Better neighbour')
        subset = random_neighbour
        best_subset = subset
    else:
        r = random.uniform(0, 1)
        # S' - S / T
        val = abs(sum(random_neighbour) - sum(subset)) / float(initial_temp)
        # print(f"val: {val}")
        if r < math.exp(-val):
            subset = random_neighbour
    
    iterations += 1
    initial_temp = cooling_ratio * initial_temp
    
    return simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations)

### Local search algorithm with heuristic on rooms - packet distribution

In [235]:
# Function to generate a single random successor state from a given  state .Is called by the generate_1000 function
def generate_one(st, packet_vec):
    st_size = len(st)
    for i in packet_vec:
        while True:
            loc=np.random.randint(0, st_size)
            if st[loc] >= i:
                st[loc] = st[loc] - i
                break
    return st


# Generates 10000 successors and returns the unique successors calls the generate_one 10000 times
def generate_10000(state,packet_vec):
    a = []
    for i in range(10000):
        a.append(generate_one(state.copy(),packet_vec))
    return set([tuple(i) for i in a]) ## only returns unique states 


# Finds the best state based on the generated random states and the free_space, chooses the one which optimises the most 
# effective space consumed

def best_State(state_collection, rooms, updated_room_state):
    cost = 9999
    no_of_room_changes = 9999
    bigger_rooms = -1
    
    initial = rooms # [5, 28, 20, 46, 25, 35] # 
    for curr_state in state_collection:
        free_space = sum(np.ones(len(rooms), dtype = int) - ((np.array(initial) - np.array(curr_state)) / np.array(initial)))
        diff = np.array(updated_room_state) - np.array(curr_state)
        # This will give you how many rooms changed
        diff_arr = np.where(diff > 0)[0]
        rooms_changed = len(diff_arr)
        # This gives what is the cost of bigger rooms selected
        room_cost = sum(diff_arr)
        if cost > free_space and rooms_changed < no_of_room_changes and room_cost > bigger_rooms:
            print(updated_room_state, rooms_changed, curr_state)
            locally_optimal_state = curr_state
            no_of_room_changes = rooms_changed
            bigger_rooms = room_cost
        
    return locally_optimal_state


def local_Search_Rooms(packet_string, current_state, rooms):
    all_generated_successors = generate_10000(current_state, packet_string)
    best_state = best_State(all_generated_successors, rooms, current_state)
    return best_state

### Packet Delivery Agent driver code

In [236]:
# Helper method to generate list from weight map
def generate_list_from_weight_map(weight_map):
    final = []
    for key in weight_map.keys():
        if weight_map[key] != 0:
            final.extend([key] * weight_map[key])
    return final


def packet_delivery_agent(weight_map, max_weight_robot_can_carry, rooms):
    total_no_of_commutes = 0
    rooms.sort()
    
    # Constants for simulated annealing
    max_iterations = 500
    cooling_ratio = 0.99
    final_temp = 0.01
    old_temp = 1500
    
    target_sum = max_weight_robot_can_carry # For example:- 10
    
    room_states = rooms
    
    while(len(list(filter(lambda x: weight_map[x] > 0, list(weight_map.keys())))) > 0):
        weights = generate_list_from_weight_map(weight_map)

        # Generating a random subset
        subset = generate_random_solution(weights, random.randint(1, len(weights)))
        best_subset = subset
        
        temp_subset = subset
        temp_best_subset = subset
        
        iterations = 1
        
        initial_temp = 1500
        
        final_subset = simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations)

        # Robot should carry maximum of 10kgs
        if(sum(final_subset) > 10):
            final_subset = simulated_annealing(target_sum, weights, temp_subset, temp_best_subset, old_temp, final_temp, max_iterations, cooling_ratio, iterations)

        print(f"Final solution: {final_subset} and sum: {sum(final_subset)}")
        updated_state = local_Search_Rooms(final_subset, room_states, rooms)
        print(f"Updated state: {updated_state}")

        for item in final_subset:
            weight_map[item] -= 1

        room_states = list(updated_state)

        print(f"Updated weight map is: {weight_map}")
        total_no_of_commutes += 1
        print(f"--------Commutes count : {total_no_of_commutes}-----------")
        # TODO: Do we need a return type

### Code Usage and Run

In [237]:
##########################################################################
"""
    Packet Delivery Agent code Usage
________________________________________

1. Initialize your weight map 
   For Example:- weight_map = {1: 2, 5: 2, 4: 2, 6: 2, 9: 2}
2. Decide on the maximum weight that the robot can carry in kgs
   For Example:- max_weight_robot_can_carry = 10
3. Decide on the rooms that are present
   For Example:- rooms = [25, 45, 36] 

4. Finally call the packet_delivery_agent method like this:-
   packet_delivery_agent(weight_map, max_weight_robot_can_carry, rooms)
"""
##########################################################################

# weight_map = {7: 2, 9: 2}
# Code run goes here:- 
rooms = [5, 28, 20, 46, 25, 35]
weight_map = {1: 15, 2: 5, 4: 7, 5: 2, 6: 3, 7: 2, 9: 4}

packet_delivery_agent(weight_map, 10, rooms)

Final solution: [5, 1, 4] and sum: 10
[5, 20, 25, 28, 35, 46] 3 (1, 15, 25, 28, 35, 45)
[5, 20, 25, 28, 35, 46] 2 (5, 20, 25, 28, 31, 40)
Updated state: (5, 20, 25, 28, 31, 40)
Updated weight map is: {1: 14, 2: 5, 4: 6, 5: 1, 6: 3, 7: 2, 9: 4}
--------Commutes count : 1-----------
Final solution: [4, 5, 1] and sum: 10
[5, 20, 25, 28, 31, 40] 3 (5, 16, 24, 23, 31, 40)
[5, 20, 25, 28, 31, 40] 2 (5, 20, 25, 22, 31, 36)
Updated state: (5, 20, 25, 22, 31, 36)
Updated weight map is: {1: 13, 2: 5, 4: 5, 5: 0, 6: 3, 7: 2, 9: 4}
--------Commutes count : 2-----------
Final solution: [7, 2, 1] and sum: 10
[5, 20, 25, 22, 31, 36] 3 (5, 13, 25, 20, 31, 35)
Updated state: (5, 13, 25, 20, 31, 35)
Updated weight map is: {1: 12, 2: 4, 4: 5, 5: 0, 6: 3, 7: 1, 9: 4}
--------Commutes count : 3-----------
Final solution: [4, 6] and sum: 10
[5, 13, 25, 20, 31, 35] 2 (5, 13, 25, 20, 25, 31)
Updated state: (5, 13, 25, 20, 25, 31)
Updated weight map is: {1: 12, 2: 4, 4: 4, 5: 0, 6: 2, 7: 1, 9: 4}
--------Commu